# Optiver p-player interval-selection game

Original problem statement: Three players A, B, C play the following game. First, A picks a real number between 0 and 1 (both inclusive), then B picks a number in the same range (different from A’s choice) and finally C picks a number, also in the same range, (different from the two chosen numbers). We then pick a number in the range uniformly randomly. Whoever’s number is closest to this random number wins the game. Assume that A, B and C all play optimally and their sole goal is to maximise their chances of winning. Also assume that if one of them has several optimal choices, then that player will randomly pick one of the optimal choices.

This ipynb file gives an algorithm for computing the optimal choice for player i in a p-player game (for i ≠ 0, we assume to know the choice of players 0, 1, ..., i-1).
This ipynb file should be read in conjunction with the submitted puzzle solution, as the technical details are explained therein.

We shall refer to the players as players 0, 1, 2, ..., p-1. Player 0 makes their selection first, player 1 second, and so forth.
Throughout this document, we shall use n to denote the discretization of the interval [0,1], where we work in the discrete interval {0, 1, ..., n-1}.
In what follows, the 'adjusted values' refer to the values chosen by players, divided by n-1 so that the choices are mapped back to the [0,1] interval. For example, if player 0 chooses the value 4 for n = 10, their adjusted value is 4/9 = 0.44.

In [1]:
import numpy as np
from operator import add

# Computing probabilities

As a starting point, we require a function that computes the win probability of each player if their choices are known.

In [2]:
def compute_probabilities(choices, n):
    """
    Returns the win probability of each player given their choices and the interval discretization.
    Note that the function can take inputs for any number of players.
    
    Inputs:
    :param choices: A list of player choices. Each number in the list must be distinct, and they must 
                    fall in the range from 0 to n-1. Index i of the list corresponds to the choice of player i.
    :param n: The discretization of the interval [0,1] into {0,1,...,n-1}.
    
    Output:
    A list of player win probabilities, with the index i of the list corresponding to player i.
    """
    
    # Initialize the probability list.
    probabilities = [0]*len(choices)
    
    # A list of players (in integers) ordered by increasing choices.
    # The list describes how the players would be ordered in the interval {0,1,...,n-1}.
    sorted_choices = sorted(range(len(choices)), key=lambda k:choices[k])
    
    for i in range(len(choices)):
        # For every player i:
        if sorted_choices.index(i) == 0:
            # if player i chose the smallest number, their win probability is decided by their one neighbour.
            probabilities[i] = (choices[i] + ((choices[sorted_choices[1]] - choices[i]) / 2))/(n-1)
        elif sorted_choices.index(i) == len(choices)-1:
            # else if player i chose the largest number, their win probability is decided by their one neighbour.
            probabilities[i] = (n-1 - choices[i] + ((choices[i] - choices[sorted_choices[-2]]) / 2))/(n-1)
        else:
            # else their win probability is decided by their two neighbours.
            probabilities[i] = ((choices[sorted_choices[sorted_choices.index(i)+1]] 
                                 - choices[sorted_choices[sorted_choices.index(i)-1]])/2)/(n-1)
            
    return probabilities

In [3]:
n = 10
choices = [6, 2, 7, 4]
adjusted_choices = [round((x/(n-1)), 2) for x in choices]

print(f'Discretization value (n)\t\t\t: {n}')
print(f'Number of players (p)\t\t\t\t: {len(choices)}')
print(f'Values chosen by players in {[x for x in range(len(choices))]}\t: {choices}')
print(f'Adjusted chosen values\t\t\t\t: {adjusted_choices}\n')

probabilities = compute_probabilities(choices, n)

for i, probability in enumerate(probabilities):
    print(f'Win probability of player {i}\t\t\t: {probability:.2f}')

Discretization value (n)			: 10
Number of players (p)				: 4
Values chosen by players in [0, 1, 2, 3]	: [6, 2, 7, 4]
Adjusted chosen values				: [0.67, 0.22, 0.78, 0.44]

Win probability of player 0			: 0.17
Win probability of player 1			: 0.33
Win probability of player 2			: 0.28
Win probability of player 3			: 0.22


# Calculating expected win probability when faced with multiple optimal choices

Players choose their values uniform randomly when faced with multiple optimal choices. This means that player i's expected win probability depends on player j's choices for all j > i. 
Therefore, faced with multiple choices, we need a function that calculates the expected win probability for all players.
The following function calculates the component-wise average for a given list of lists.

In [4]:
def average_sum_of_lists(scores):
    '''
    Returns the component-wise average for a given list of lists.
    
    Inputs:
    :param scores: A list of lists, where each nested list is of the same size, and contain float elements.
    
    Output:
    A list whose i'th element is the mean of the i'th elements within the input nested lists.
    '''
    
    # Initialize the output list.
    average = [0]*len(scores[0])

    # Adds every nested list in scores component-wise.
    for item in scores:
        average = list(map(add, average, item))
        
    # Take the mean of all elements in the list average, by dividing by the number of nested lists of input. 
    average = [number / len(scores) for number in average]
    return average

In [5]:
scores = [[0.06, 0.39, 0.27, 0.28], [0.11, 0.39, 0.22, 0.28], [0.17, 0.39, 0.16, 0.28], [0.22, 0.39, 0.11, 0.28]]
average_scores = average_sum_of_lists(scores)

print(f'Input\t : {scores}')
print(f'Output\t : {[round(item, 2) for item in average_scores]}')

Input	 : [[0.06, 0.39, 0.27, 0.28], [0.11, 0.39, 0.22, 0.28], [0.17, 0.39, 0.16, 0.28], [0.22, 0.39, 0.11, 0.28]]
Output	 : [0.14, 0.39, 0.19, 0.28]


# Computing the expected win probability

We now have the required ingredients to compute the expected win probability for each player. 
The following function returns the optimal choice of player i in a p player interval selection game, given the choices of players 0, 1, ..., i (for our purposes, we are interested in the case p = 4, but more on this later).

The following recursive function creates a new branch for every possible choice that players can make, thereby constructing a depth-p tree. At each node of the tree, the optimal expected win probability can only be computed if that of its children have been computed. At depth-p of the tree, there are no more children to recurse on, as the game is played by p players. At this point, one may compute the expected win probability for each player since all choices are known. 

We give three in the cells thereafter to illustrate how this function works.

In [6]:
def compute_expected_values(n, p, i=0, history = None):
    '''
    Returns the optimal choice of player i that maximizes their expected win probability given the choice 
    of all players before player i.
    
    Inputs:
    :param n: Discretization of the interval [0,1] into {0,1,...,n-1}.
    :param p: Number of players in the game.
    :param i: Player i.
    :param history: A list of choices made by players before player i. For example, history[j] is the choice of player j.
    
    Output:
    A dictionary with elements (k,v) where k is a tuple and v is a list. 
    k[i] denotes the optimal choice for player i, and k[j] is the element chosen by player j, i.e. k[j] = history[j].
    k is always of length i.
    The value v is a list of expected win probabilities for the p players given the choices in k. 
    '''
    
    # If the history is not specified, set history = [] as default.
    if history is None:
        history = []
    
    # If player 0 is chosen, we can search for their optimal choice in {0,1,...,floor(n/2)} by symmetry.
    # Else, we want to ensure that player i does not choose a value that has already been chosen.
    if i == 0:
        options = [x for x in range(int(np.floor((n/2))))]
    else:
        options = [x for x in range(n) if x not in history]
        
    # Initialize the dictionary selection, which will be the output.
    # Initialize the list choices, which will keep track of the number selected by player i.
    selection = {}
    choices = []
    
    for option in options:
        # For every possible choice for player i, 
        # update the list of choices made so far
        choices = history + [option]
        if i == p-1:
            # if player i goes last, then we have all choices made by players and hence the expected
            # win probability is precisely the win probability. We store this information in the dictionary `selection'
            # as a value for the key `choices', where choices is a list of all choices made.
            selection[tuple(choices)] = compute_probabilities(choices, n)
        else:
            # else, player i does not go last. Then their expected win probability depends on the choice
            # of the players that go after them. Because of this, compute the optimal solutions chosen
            # by the players after.
            
            # Update the list that keeps track of choices made so far, and find optimal choices for player i+1. 
            value = compute_expected_values(n, p, i+1, choices)
            
            # From the optimal choices of player i+1, compute the expected win probability for each of the players 
            # 0,1,...,p. Keep this information in the dictionary `selection' as a value for the key `choices', where 
            # the choices is a list keeping track of choices made by players 0,1,...,i.
            scores = [probability for x,probability in value.items()]
            selection[tuple(choices)] = average_sum_of_lists(scores)
    
    # Compute the choices for player i that maximizes their expected win probability, and only output such choices 
    # (together with their probability).
    opt_ev = max([x[i] for x in list(selection.values())])
    return {k:v for (k,v) in selection.items() if v[i] >= opt_ev}

## Examples

We give three examples to illustrate compute_expected_values.

### Example 1

- 10 number discretization of [0,1];
- 4 players;
- player 0 has chosen 4;

Compute optimal choice for player 1.

In [7]:
n = 10
p = 4
i = 1
history = [4]
adjusted_history = [round((x/(n-1)), 2) for x in history]

score = compute_expected_values(n, p, i, history)
print(f'Discretization value (n)\t\t: {n}')
print(f'Number of players (p)\t\t\t: {p}')
print(f'Values chosen by players in {[x for x in range(i)]}\t\t: {history}')
print(f'Adjusted chosen values\t\t\t: {adjusted_history}\n')

for k, v in score.items():
    print(f'Optimal choice for player 1\t\t: {k[1]}')
    print(f'Adjusted optimal choice for player 1\t: {(k[1]/(n-1)):.2f}\n')
    for i in range(4):
        # Round all expected win probabilities to hundredth place.
        print(f'Expected win probability for player {i}\t: {v[i]:.2f}')

Discretization value (n)		: 10
Number of players (p)			: 4
Values chosen by players in [0]		: [4]
Adjusted chosen values			: [0.44]

Optimal choice for player 1		: 7
Adjusted optimal choice for player 1	: 0.78

Expected win probability for player 0	: 0.24
Expected win probability for player 1	: 0.31
Expected win probability for player 2	: 0.29
Expected win probability for player 3	: 0.17


### Example 2

- 30 number discretization of [0,1];
- 4 players;
- players 0, 1, 2 have chosen the values 5, 24, 18, respectively.

Compute the optimal choice for player 3.

In this case, player 3 maximizes their chances by choosing a number from the interval (5, 18). As we have seen from the 3-player variant of the game, it does not matter what value they choose within the interval. This is reflected below as all choices from and including 6 to and including 17 yield the same optimal win probability for player 3.

Note that because all four choices of the players are determined at this point, the expected win probability (value) for each of the choices (key) is precisely the win probability.

In [8]:
n = 30
p = 4
i = 3
history = [5, 24, 18]
adjusted_history = [round((x/(n-1)), 2) for x in history]

print(f'Discretization value (n)\t\t: {n}')
print(f'Number of players (p)\t\t\t: {p}')
print(f'Values chosen by players in {[x for x in range(i)]}\t: {history}')
print(f'Adjusted chosen values\t\t\t: {adjusted_history}\n')

score = compute_expected_values(n, p, i, history)

optimal_choice_p3 = [x[i] for x in score.keys()]
expected_win_probability = [x for x in score.values()]

print(f'Optimal choices for player {i}\t\t: {optimal_choice_p3}')
print(f'Adjusted optimal choices for player {i}\t: {[round((x/(n-1)), 2) for x in optimal_choice_p3]}')
print(f'Expected win probability for player {i}\t: {expected_win_probability[0][i]:.2f}')

Discretization value (n)		: 30
Number of players (p)			: 4
Values chosen by players in [0, 1, 2]	: [5, 24, 18]
Adjusted chosen values			: [0.17, 0.83, 0.62]

Optimal choices for player 3		: [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
Adjusted optimal choices for player 3	: [0.21, 0.24, 0.28, 0.31, 0.34, 0.38, 0.41, 0.45, 0.48, 0.52, 0.55, 0.59]
Expected win probability for player 3	: 0.22


### Example 3

- 50 number discretization of [0,1];
- 3 players;
- player 0 has chosen the value 0.

Compute the optimal choice for player 1.

Note that this solves question 1 of the original 3-player game, where player 0 chooses the value 0. We expect player 1 to choose 2/3.

In [9]:
n = 40
p = 3
i = 1
history = [0]
adjusted_history = [round((x/(n-1)), 2) for x in history]

score = compute_expected_values(n, p, i, history)

print(f'Discretization value (n)\t\t: {n}')
print(f'Number of players (p)\t\t\t: {p}')
print(f'Values chosen by players in {[x for x in range(i)]}\t\t: {history}')
print(f'Adjusted chosen values\t\t\t: {adjusted_history}\n')

for k, v in score.items():
    print(f'Optimal choice for player {i}\t\t: {k[1]}')
    print(f'Adjusted optimal choice for player {i}\t: {(k[1]/(n-1)):.2f}')
    for i in range(3):
        # Round all expected win probabilities to hundredth place.
        print(f'Expected win probability for player {i}\t: {v[i]:.2f}')

Discretization value (n)		: 40
Number of players (p)			: 3
Values chosen by players in [0]		: [0]
Adjusted chosen values			: [0.0]

Optimal choice for player 1		: 26
Adjusted optimal choice for player 1	: 0.67
Expected win probability for player 0	: 0.17
Expected win probability for player 1	: 0.50
Expected win probability for player 2	: 0.33


# Computing the optimal choice of player 0 in a 4-player game

Lastly, to determine the optimal choice of player 0 in the interval [0,1] for a 4-player game, we run the function compute_expected_values on multiple discretization values (20 <= n < 40, chosen arbitrarily). During each iteration, it is possible that player 0 has multiple optimal choices. In that case we take the median of those optimal choices; otherwise, we take the optimal choice. This output is divided by n-1 and stored into a list. Finally, we take the median of the list to estimate an optimal choice for player 0 in the interval [0,1]. In particular, we take the median to account for any outliers that may appear. In the output below, oc and ewp refer to the optimal choice and the expected win probability of player 0.

In [10]:
# Initialize the output lists and specify the number of players p.
optimal = []
probability = []
p = 4

for n in range(20, 40):
    # For every discretization value n,
    # Initialize a list to keep track of player 0's optimal choices.
    optimal_a = []
    
    # Compute the optimal choice of player 0 for n.
    optimal_score = compute_expected_values(n, p)
    
    # For every optimal choice of player 0, divide the choice by n-1 to obtain a value in [0,1], and store these in a list.
    optimal_a = [item[0]/(n-1) for item in optimal_score]
    
    # Store the median value into the global list optimal.
    optimal.append(np.median(optimal_a))
    
    # Store every value of dictionary optimal_score in a list.
    optimal_prob = [v for v in optimal_score.values()]
    
    # Store the expected win probability for player 0 in a global list.
    probability.append(optimal_prob[0][0])
    
    # Print the optimal choice (oc) and expected win probability (ewp) of player 0.
    print(f'n = {n}\t oc : {[round(item,2) for item in optimal_a]}\t ewp : {optimal_prob[0][0]:.2f}')          

print(f'\n The estimated optimal choice for player 0 in the interval [0,1] is {np.median(optimal)}, and the median expected win probability is {np.median(probability):.2f}.')

n = 20	 oc : [0.16]	 ewp : 0.28
n = 21	 oc : [0.15]	 ewp : 0.28
n = 22	 oc : [0.19]	 ewp : 0.27
n = 23	 oc : [0.18]	 ewp : 0.27
n = 24	 oc : [0.17]	 ewp : 0.29
n = 25	 oc : [0.17]	 ewp : 0.29
n = 26	 oc : [0.16]	 ewp : 0.29
n = 27	 oc : [0.15]	 ewp : 0.28
n = 28	 oc : [0.19]	 ewp : 0.29
n = 29	 oc : [0.18]	 ewp : 0.29
n = 30	 oc : [0.17]	 ewp : 0.29
n = 31	 oc : [0.17]	 ewp : 0.29
n = 32	 oc : [0.16]	 ewp : 0.29
n = 33	 oc : [0.16]	 ewp : 0.28
n = 34	 oc : [0.18]	 ewp : 0.30
n = 35	 oc : [0.18]	 ewp : 0.26
n = 36	 oc : [0.14]	 ewp : 0.27
n = 37	 oc : [0.17]	 ewp : 0.29
n = 38	 oc : [0.16]	 ewp : 0.29
n = 39	 oc : [0.16]	 ewp : 0.29

 The estimated optimal choice for player 0 in the interval [0,1] is 0.16666666666666666, and the median expected win probability is 0.29.


# Computing the optimal choice of player 0 in a 3-player game

Because the function compute_expected_values generalizes to p players, we may in particular estimate the optimal choice of player 0 in the interval [0,1] for a 3-player game. We employ the same procedure as done for the 4-player variant.

We see that this does indeed confirm our optimal choice for player 0 of 0.25 (and 0.75), together with their expected win probability of around 3/8. Using notation from the above cell, oc and ewp refer now to the optimal choice(s) and the expected win probability for player 2.

In [11]:
# Initialize the output lists and specify the number of players p.
optimal = []
probability = []
p = 3

for n in range(30,50):
    # For every discretization value n,
    # Initialize a list to keep track of player 0's optimal choices.
    optimal_a = []
    
    # Compute the optimal choice of player 0 for n.
    optimal_score = compute_expected_values(n, p)
    
    # For every optimal choice of player 0, divide the choice by n-1 to obtain a value in [0,1], and store these in a list.
    optimal_a = [item[0]/(n-1) for item in optimal_score]
    
    # Store the median value into the global list optimal.
    optimal.append(np.median(optimal_a))
    
    # Store every value of dictionary optimal_score in a list.
    optimal_prob = [v for v in optimal_score.values()]
    
    # Store the expected win probability for player 0 in a global list.
    probability.append(optimal_prob[0][0])
    
    # Print the optimal choice (oc) and expected win probability (ewp) of player 0.
    print(f'n = {n}\t oc : {[round(item,2) for item in optimal_a]}\t ewp : {optimal_prob[0][0]:.2f}')          

print(f'\n The estimated optimal choice for player 0 in the interval [0,1] is {np.median(optimal):.4f}, and the median expected win probability is {np.median(probability):.2f}.')

n = 30	 oc : [0.24]	 ewp : 0.37
n = 31	 oc : [0.27]	 ewp : 0.38
n = 32	 oc : [0.26]	 ewp : 0.38
n = 33	 oc : [0.25]	 ewp : 0.38
n = 34	 oc : [0.24]	 ewp : 0.37
n = 35	 oc : [0.26]	 ewp : 0.38
n = 36	 oc : [0.26]	 ewp : 0.38
n = 37	 oc : [0.25]	 ewp : 0.37
n = 38	 oc : [0.24]	 ewp : 0.37
n = 39	 oc : [0.26]	 ewp : 0.38
n = 40	 oc : [0.26]	 ewp : 0.38
n = 41	 oc : [0.25]	 ewp : 0.38
n = 42	 oc : [0.24]	 ewp : 0.37
n = 43	 oc : [0.26]	 ewp : 0.38
n = 44	 oc : [0.26]	 ewp : 0.38
n = 45	 oc : [0.25]	 ewp : 0.37
n = 46	 oc : [0.24]	 ewp : 0.37
n = 47	 oc : [0.26]	 ewp : 0.38
n = 48	 oc : [0.26]	 ewp : 0.38
n = 49	 oc : [0.25]	 ewp : 0.38

 The estimated optimal choice for player 0 in the interval [0,1] is 0.2527, and the median expected win probability is 0.38.
